In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F

ModuleNotFoundError: No module named 'pandas'

In [1]:
!pip install -q torch torchvision pandas

In [ ]:
# Creating synthetic image data for demonstration
import numpy as np

# Function to create a synthetic image with marked cores
def create_synthetic_image(cores_coordinates):
    image = np.zeros((1024, 1024, 3), dtype=np.uint8)
    for x, y in cores_coordinates:
        image[y-5:y+5, x-5:x+5] = [255, 0, 0]  # Marking the core with red color
    return image

# Updating the sample dataset with synthetic images
for item in sample_dataset:
    item['image'] = create_synthetic_image(item['cores'])

# Modifying the dataset class to use synthetic images
class ModifiedTissueMicroarrayDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image = item['image']
        cores = item['cores']

        if self.transform:
            image = self.transform(image)

        return image, cores

# Creating data loaders with the modified dataset class
train_loader = DataLoader(ModifiedTissueMicroarrayDataset(train_data), batch_size=2, shuffle=True)
valid_loader = DataLoader(ModifiedTissueMicroarrayDataset(valid_data), batch_size=2, shuffle=False)

# Printing out one batch of data
for images, cores in train_loader:
    print(images.shape)
    print(cores)

torch.Size([1, 1024, 1024, 3])
[[tensor([50]), tensor([60])], [tensor([150]), tensor([160])], [tensor([250]), tensor([260])]]


In [ ]:
# Load and Preprocess the Dataset
# For the purpose of this example, we'll create a small sample dataset
# In practice, you should load your actual dataset here

# Sample dataset: list of dictionaries containing image file paths and core coordinates
sample_dataset = [
    {'image_path': 'image1.png', 'cores': [(50, 60), (150, 160), (250, 260)]},
    {'image_path': 'image2.png', 'cores': [(75, 85), (175, 185), (275, 285)]},
    # Add more data as needed
]

# Visualize the sample data
for data in sample_dataset:
    print(f"Image Path: {data['image_path']}")
    print(f"Cores Coordinates: {data['cores']}")

Image Path: image1.png
Cores Coordinates: [(50, 60), (150, 160), (250, 260)]
Image Path: image2.png
Cores Coordinates: [(75, 85), (175, 185), (275, 285)]


In [ ]:
# Prepare the Dataset for Training

# Creating a custom dataset class
class TissueMicroarrayDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image = plt.imread(item['image_path'])  # This is a placeholder; replace with actual image loading
        cores = item['cores']

        if self.transform:
            image = self.transform(image)

        return image, cores

# Splitting the dataset into training and validation sets
train_data = sample_dataset[:int(len(sample_dataset)*0.8)]  # 80% for training
valid_data = sample_dataset[int(len(sample_dataset)*0.8):]  # 20% for validation

# Creating data loaders
train_loader = DataLoader(TissueMicroarrayDataset(train_data), batch_size=2, shuffle=True)
valid_loader = DataLoader(TissueMicroarrayDataset(valid_data), batch_size=2, shuffle=False)

# Printing out one batch of data
for images, cores in train_loader:
    print(images)
    print(cores)

FileNotFoundError: [Errno 2] No such file or directory: 'image1.png'

In [ ]:
# Choose a Pre-trained Object Detection Model

# Loading a pre-trained Faster R-CNN model
model = fasterrcnn_resnet50_fpn(pretrained=True)

# Modifying the model to fit our specific use case
num_classes = 2  # 1 class (core) + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Printing the modified model architecture
print(model)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /home/noteable/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

 21%|██▏       | 34.3M/160M [00:00<00:00, 359MB/s]

 43%|████▎     | 68.5M/160M [00:00<00:00, 326MB/s]

 62%|██████▏   | 99.8M/160M [00:00<00:00, 303MB/s]

 84%|████████▍ | 135M/160M [00:00<00:00, 326MB/s] 

100%|██████████| 160M/160M [00:00<00:00, 332MB/s]

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
# Fine-tuning the Model

from torch.optim import SGD
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torch.utils.data import random_split

# Defining the training function
def train_model(model, data_loader, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for images, targets in data_loader:
            # Move the images and targets to the GPU if available
            images = list(image.to('cuda') for image in images)
            targets = [{k: v.to('cuda') for k, v in t.items()} for t in targets]

            # Forward pass
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

            # Backward pass and optimization
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

            total_loss += losses.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(data_loader)}')

# Move the model to the GPU if available
model.to('cuda')

# Defining the optimizer
optimizer = SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)

# Training the model
# Note: We should replace the 'train_loader' with the actual data loader of your dataset
train_model(model, train_loader, optimizer, num_epochs=10)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
# Fine-tuning the Model on CPU

# Modifying the training function to run on CPU
def train_model_cpu(model, data_loader, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for images, targets in data_loader:
            # Forward pass
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

            # Backward pass and optimization
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

            total_loss += losses.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(data_loader)}')

# Move the model back to CPU
model.to('cpu')

# Training the model on CPU
# Note: We should replace the 'train_loader' with the actual data loader of your dataset
train_model_cpu(model, train_loader, optimizer, num_epochs=10)

NameError: name 'optimizer' is not defined

In [ ]:
# Redefining the optimizer and proceeding with the training
from torch.optim import SGD

# Defining the optimizer
optimizer = SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)

# Training the model on CPU
train_model_cpu(model, train_loader, optimizer, num_epochs=10)

TypeError: list indices must be integers or slices, not str

In [ ]:
# Correcting the target format and proceeding with the training

# Modifying the dataset class to provide targets in the correct format
class CorrectedTissueMicroarrayDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image = item['image']
        cores = item['cores']

        # Creating bounding boxes around the core coordinates
        boxes = [[x-5, y-5, x+5, y+5] for x, y in cores]
        boxes = torch.tensor(boxes, dtype=torch.float32)

        # Creating labels for the cores
        labels = torch.ones((len(cores),), dtype=torch.int64)

        target = {'boxes': boxes, 'labels': labels}

        if self.transform:
            image = self.transform(image)

        return image, target

# Creating data loaders with the corrected dataset class
train_loader = DataLoader(CorrectedTissueMicroarrayDataset(train_data), batch_size=2, shuffle=True)
valid_loader = DataLoader(CorrectedTissueMicroarrayDataset(valid_data), batch_size=2, shuffle=False)

# Training the model on CPU with the corrected targets
train_model_cpu(model, train_loader, optimizer, num_epochs=10)

TypeError: string indices must be integers

In [ ]:
# Correcting the 'cores' key access issue and proceeding with the training

# Modifying the dataset class to correctly access the 'cores' key
class FinalTissueMicroarrayDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image = item['image']
        cores = item['cores']

        # Creating bounding boxes around the core coordinates
        boxes = [[x-5, y-5, x+5, y+5] for x, y in cores]
        boxes = torch.tensor(boxes, dtype=torch.float32)

        # Creating labels for the cores
        labels = torch.ones((len(cores),), dtype=torch.int64)

        target = {'boxes': boxes, 'labels': labels}

        if self.transform:
            image = self.transform(image)

        return image, target

# Creating data loaders with the final dataset class
train_loader = DataLoader(FinalTissueMicroarrayDataset(train_data), batch_size=2, shuffle=True)
valid_loader = DataLoader(FinalTissueMicroarrayDataset(valid_data), batch_size=2, shuffle=False)

# Training the model on CPU with the corrected targets
train_model_cpu(model, train_loader, optimizer, num_epochs=10)

TypeError: string indices must be integers